In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the LSTM model
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for LSTM model
hyperparameters = {
    'LSTM': {
        'units': [8, 32, 64, 128, 256],
        'dropout_rate': [0.0],
        'learning_rate': [0.001],
        'batch_size': [8, 12]
    }
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics
eval_metrics = {}

# Train and evaluate the LSTM model with hyperparameter tuning
best_rmse = np.inf
best_model = None
best_units = 0
best_dropout_rate = 0
best_learning_rate = 0
best_batch_size = 0

for units in hyperparameters['LSTM']['units']:
    for dropout_rate in hyperparameters['LSTM']['dropout_rate']:
        for learning_rate in hyperparameters['LSTM']['learning_rate']:
            for batch_size in hyperparameters['LSTM']['batch_size']:
                model = create_lstm_model(units, dropout_rate)
                model.optimizer.lr = learning_rate

                start_time = time.time()
                history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                    y_train,
                                    epochs=100,
                                    batch_size=batch_size,
                                    verbose=0)
                training_time = time.time() - start_time

                predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                mse = mean_squared_error(y_test, predictions)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(y_test, predictions)
                r2 = r2_score(y_test, predictions)
                smape_val = smape(y_test, predictions)

                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = model
                    best_units = units
                    best_dropout_rate = dropout_rate
                    best_learning_rate = learning_rate
                    best_batch_size = batch_size

                print(f"LSTM (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                print(f"RMSE: {rmse:.5f}")
                print(f"MAE: {mae:.5f}")
                print(f"R-squared: {r2:.5f}")
                print(f"SMAPE: {smape_val:.5f}")
                print(f"Training Time: {training_time:.5f} seconds")
                print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                print()

# Store the predictions in the dictionary
model_data[f"LSTM (Units: {best_units}, Dropout Rate: {best_dropout_rate}, Learning Rate: {best_learning_rate}, Batch Size: {best_batch_size})"] = best_model.predict(
    X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

# Include evaluation metrics and best hyperparameters in the eval metrics dictionary
eval_metrics['LSTM'] = {
    'Best Units': best_units,
    'Best Dropout Rate': best_dropout_rate,
    'Best Learning Rate': best_learning_rate,
    'Best Batch Size': best_batch_size,
    'RMSE': best_rmse,
    'MAE': mae,
    'R-squared': r2,
    'SMAPE': smape_val,
    'Training Time (seconds)': training_time,
    'CPU Usage (MHz)': psutil.cpu_percent(),
    'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
}

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('lstm_forecasts_1875_to_1941_792interval.csv', index=False)

# Save evaluation metrics to a CSV file
df_metrics = pd.DataFrame(eval_metrics).T
df_metrics.to_csv('lstm_eval_metrics_1875_to_1941_792interval.csv')

# Print true values and forecasts in table format
df_true_values = pd.DataFrame(y_test, columns=['True Value'])
df_forecasts = pd.DataFrame(model_data)

print("True Values and Forecasts:")
print(pd.concat([df_true_values, df_forecasts], axis=1))


3/3 [==============================] - 0s 7ms/step
LSTM (Units: 8, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 95.79604
MAE: 86.35425
R-squared: -3.82686
SMAPE: 0.86176
Training Time: 28.85538 seconds
CPU Usage: 21.1 MHz
Memory Used: 9023.625 MB

3/3 [==============================] - 0s 3ms/step
LSTM (Units: 8, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 12)
RMSE: 105.85124
MAE: 96.80366
R-squared: -4.89334
SMAPE: 1.01179
Training Time: 19.44170 seconds
CPU Usage: 27.6 MHz
Memory Used: 9038.45703125 MB

3/3 [==============================] - 0s 3ms/step
LSTM (Units: 32, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 38.39452
MAE: 29.46473
R-squared: 0.22463
SMAPE: 0.40429
Training Time: 25.93659 seconds
CPU Usage: 28.5 MHz
Memory Used: 9142.171875 MB

3/3 [==============================] - 0s 3ms/step
LSTM (Units: 32, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 12)
RMSE: 56.26533
MAE: 45.74767
R-squared: -0.66514
SMAPE: 0.47421
Traini

In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the LSTM model
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for LSTM model
hyperparameters = {
    'LSTM': {
        'units': [8, 32, 64, 128, 256],
        'dropout_rate': [0.0, 0.2],
        'learning_rate': [0.001, 0.005],
        'batch_size': [8]
    }
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics
eval_metrics = {}

# Train and evaluate the LSTM model with hyperparameter tuning
best_rmse = np.inf
best_model = None
best_units = 0
best_dropout_rate = 0
best_learning_rate = 0
best_batch_size = 0

for units in hyperparameters['LSTM']['units']:
    for dropout_rate in hyperparameters['LSTM']['dropout_rate']:
        for learning_rate in hyperparameters['LSTM']['learning_rate']:
            for batch_size in hyperparameters['LSTM']['batch_size']:
                model = create_lstm_model(units, dropout_rate)
                model.optimizer.lr = learning_rate

                start_time = time.time()
                history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                    y_train,
                                    epochs=100,
                                    batch_size=batch_size,
                                    verbose=0)
                training_time = time.time() - start_time

                predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                mse = mean_squared_error(y_test, predictions)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(y_test, predictions)
                r2 = r2_score(y_test, predictions)
                smape_val = smape(y_test, predictions)

                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = model
                    best_units = units
                    best_dropout_rate = dropout_rate
                    best_learning_rate = learning_rate
                    best_batch_size = batch_size

                print(f"LSTM (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                print(f"RMSE: {rmse:.5f}")
                print(f"MAE: {mae:.5f}")
                print(f"R-squared: {r2:.5f}")
                print(f"SMAPE: {smape_val:.5f}")
                print(f"Training Time: {training_time:.5f} seconds")
                print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                print()

# Store the predictions in the dictionary
model_data[f"LSTM (Units: {best_units}, Dropout Rate: {best_dropout_rate}, Learning Rate: {best_learning_rate}, Batch Size: {best_batch_size})"] = best_model.predict(
    X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

# Include evaluation metrics and best hyperparameters in the eval metrics dictionary
eval_metrics['LSTM'] = {
    'Best Units': best_units,
    'Best Dropout Rate': best_dropout_rate,
    'Best Learning Rate': best_learning_rate,
    'Best Batch Size': best_batch_size,
    'RMSE': best_rmse,
    'MAE': mae,
    'R-squared': r2,
    'SMAPE': smape_val,
    'Training Time (seconds)': training_time,
    'CPU Usage (MHz)': psutil.cpu_percent(),
    'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
}

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('lstm_forecasts_1875_to_1941_2.csv', index=False)

# Save evaluation metrics to a CSV file
df_metrics = pd.DataFrame(eval_metrics).T
df_metrics.to_csv('lstm_eval_metrics_1875_to_1941_2.csv')

# Print true values and forecasts in table format
df_true_values = pd.DataFrame(y_test, columns=['True Value'])
df_forecasts = pd.DataFrame(model_data)

print("True Values and Forecasts:")
print(pd.concat([df_true_values, df_forecasts], axis=1))


3/3 [==============================] - 1s 6ms/step
LSTM (Units: 8, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 96.08934
MAE: 86.67450
R-squared: -3.85646
SMAPE: 0.86601
Training Time: 24.78710 seconds
CPU Usage: 24.2 MHz
Memory Used: 9306.3203125 MB

3/3 [==============================] - 0s 3ms/step
LSTM (Units: 8, Dropout Rate: 0.0, Learning Rate: 0.005, Batch Size: 8)
RMSE: 30.65566
MAE: 22.76835
R-squared: 0.50570
SMAPE: 0.39251
Training Time: 28.21861 seconds
CPU Usage: 35.4 MHz
Memory Used: 9259.67578125 MB

3/3 [==============================] - 0s 6ms/step
LSTM (Units: 8, Dropout Rate: 0.2, Learning Rate: 0.001, Batch Size: 8)
RMSE: 93.76140
MAE: 84.08180
R-squared: -3.62400
SMAPE: 0.83151
Training Time: 27.16693 seconds
CPU Usage: 35.7 MHz
Memory Used: 9267.02734375 MB

3/3 [==============================] - 0s 0s/step
LSTM (Units: 8, Dropout Rate: 0.2, Learning Rate: 0.005, Batch Size: 8)
RMSE: 36.84452
MAE: 28.99686
R-squared: 0.28597
SMAPE: 0.41003
Trainin

In [ ]:
# Regression models 792 MONTHS (720 Months training, and 72 Testing) 
# Interval (1875-1941)

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests
import tensorflow as tf

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the target variable
# scaler = MinMaxScaler(feature_range=(0, 1))
# target_scaled = scaler.fit_transform(target)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

if X_test.shape[0] == 0:
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Define the regression models to be evaluated
models = [
    LinearRegression(),
    SVR(),
    AdaBoostRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor()
]

# Hyperparameter grids for the regression models
hyperparameter_grids = [
    {},
    {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.01, 0.001]},
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]},
    {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10]},
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]}
]

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate each model with hyperparameter tuning
best_model_predictions = []
best_model_smapes = []
best_model_mses = []
best_model_rmses = []
best_model_maes = []
best_model_r2s = []
training_times = []
cpu_usages = []
memory_usages = []

for model, hyperparameters in zip(models, hyperparameter_grids):
    # Create a grid search instance
    grid_search = GridSearchCV(estimator=model, param_grid=hyperparameters, scoring='neg_mean_squared_error', cv=3, n_jobs=1)

    # Fit the grid search to the training data
    start_time = time.time()
    grid_search.fit(X_train, y_train.ravel())
    training_time = time.time() - start_time

    # Get the best model and its predictions
    best_model = grid_search.best_estimator_
    best_predictions = best_model.predict(X_test).ravel()

    # Append the best predictions to the list
    best_model_predictions.append(best_predictions)

    # Compute evaluation metrics for the best model
    mse = mean_squared_error(y_test, best_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, best_predictions)
    r2 = r2_score(y_test, best_predictions)
    smape_val = smape(y_test, best_predictions)

    best_model_smapes.append(smape_val)
    best_model_mses.append(mse)
    best_model_rmses.append(rmse)
    best_model_maes.append(mae)
    best_model_r2s.append(r2)

    # Print the best parameters and evaluation metrics
    model_name = model.__class__.__name__
    cpu_usage = psutil.cpu_percent()
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"{model_name} RMSE: {rmse:.5f}")
    print(f"{model_name} MAE: {mae:.5f}")
    print(f"{model_name} R-squared: {r2:.5f}")
    print(f"{model_name} SMAPE: {smape_val:.5f}")
    print(f"{model_name} Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {cpu_usage:.1f} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    # Append training time, CPU usage, and memory usage to lists
    training_times.append(training_time)
    cpu_usages.append(cpu_usage)
    memory_usages.append(psutil.virtual_memory().used / 1024 / 1024)

# Combine the predictions of the best models
combined_predictions = np.mean(best_model_predictions, axis=0)
median_predictions = np.median(best_model_predictions, axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(best_model_predictions)
geometric_mean_predictions = gmean(best_model_predictions)

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': [model.__class__.__name__ for model in models] + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': best_model_rmses + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': best_model_maes + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': best_model_r2s + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': best_model_smapes + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': training_times + [None] * 4,
    'CPU Usage (MHz)': cpu_usages + [None] * 4,
    'Memory Used (MB)': memory_usages + [None] * 4
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('Reg_eval_metrics_792_Months_1875_to_1941.csv', index=False)


Best parameters for LinearRegression: {}
LinearRegression RMSE: 4.90207
LinearRegression MAE: 4.07230
LinearRegression R-squared: 0.98736
LinearRegression SMAPE: 0.40721
LinearRegression Training Time: 0.18539 seconds
CPU Usage: 23.8 MHz
Memory Used: 12790.30859375 MB

Best parameters for SVR: {'C': 10, 'epsilon': 0.001}
SVR RMSE: 23.42446
SVR MAE: 15.85800
SVR R-squared: 0.71139
SVR SMAPE: 0.39108
SVR Training Time: 0.89304 seconds
CPU Usage: 67.8 MHz
Memory Used: 12783.12109375 MB

Best parameters for AdaBoostRegressor: {'learning_rate': 0.1, 'n_estimators': 200}
AdaBoostRegressor RMSE: 11.26306
AdaBoostRegressor MAE: 8.48377
AdaBoostRegressor R-squared: 0.93328
AdaBoostRegressor SMAPE: 0.39472
AdaBoostRegressor Training Time: 8.03174 seconds
CPU Usage: 55.2 MHz
Memory Used: 12713.02734375 MB

Best parameters for RandomForestRegressor: {'max_depth': 5, 'n_estimators': 200}
RandomForestRegressor RMSE: 8.74848
RandomForestRegressor MAE: 6.67954
RandomForestRegressor R-squared: 0.95974


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests
import tensorflow as tf

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1854 to 1931)
filtered_data = data[(data['Year'] >= 1854) & (data['Year'] <= 1931)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the target variable
# scaler = MinMaxScaler(feature_range=(0, 1))
# target_scaled = scaler.fit_transform(target)

# Split the data into training and testing sets
train_size = 840  # Number of months for training
test_size = 84  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

if X_test.shape[0] == 0:
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Define the regression models to be evaluated
models = [
    LinearRegression(),
    SVR(),
    AdaBoostRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor()
]

# Hyperparameter grids for the regression models
hyperparameter_grids = [
    {},
    {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.01, 0.001]},
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]},
    {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10]},
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]}
]

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate each model with hyperparameter tuning
best_model_predictions = []
best_model_smapes = []
best_model_mses = []
best_model_rmses = []
best_model_maes = []
best_model_r2s = []
training_times = []
cpu_usages = []
memory_usages = []

for model, hyperparameters in zip(models, hyperparameter_grids):
    # Create a grid search instance
    grid_search = GridSearchCV(estimator=model, param_grid=hyperparameters, scoring='neg_mean_squared_error', cv=3, n_jobs=1)

    # Fit the grid search to the training data
    start_time = time.time()
    grid_search.fit(X_train, y_train.ravel())
    training_time = time.time() - start_time

    # Get the best model and its predictions
    best_model = grid_search.best_estimator_
    best_predictions = best_model.predict(X_test).ravel()

    # Append the best predictions to the list
    best_model_predictions.append(best_predictions)

    # Compute evaluation metrics for the best model
    mse = mean_squared_error(y_test, best_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, best_predictions)
    r2 = r2_score(y_test, best_predictions)
    smape_val = smape(y_test, best_predictions)

    best_model_smapes.append(smape_val)
    best_model_mses.append(mse)
    best_model_rmses.append(rmse)
    best_model_maes.append(mae)
    best_model_r2s.append(r2)

    # Print the best parameters and evaluation metrics
    model_name = model.__class__.__name__
    cpu_usage = psutil.cpu_percent()
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"{model_name} RMSE: {rmse:.5f}")
    print(f"{model_name} MAE: {mae:.5f}")
    print(f"{model_name} R-squared: {r2:.5f}")
    print(f"{model_name} SMAPE: {smape_val:.5f}")
    print(f"{model_name} Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {cpu_usage:.1f} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    # Append training time, CPU usage, and memory usage to lists
    training_times.append(training_time)
    cpu_usages.append(cpu_usage)
    memory_usages.append(psutil.virtual_memory().used / 1024 / 1024)

# Combine the predictions of the best models
combined_predictions = np.mean(best_model_predictions, axis=0)
median_predictions = np.median(best_model_predictions, axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(best_model_predictions)
geometric_mean_predictions = gmean(best_model_predictions)

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': [model.__class__.__name__ for model in models] + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': best_model_rmses + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': best_model_maes + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': best_model_r2s + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': best_model_smapes + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': training_times + [None] * 4,
    'CPU Usage (MHz)': cpu_usages + [None] * 4,
    'Memory Used (MB)': memory_usages + [None] * 4
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('Reg_eval_metrics_924_Months_1854_to_1931.csv', index=False)


Best parameters for LinearRegression: {}
LinearRegression RMSE: 3.62732
LinearRegression MAE: 2.85071
LinearRegression R-squared: 0.98806
LinearRegression SMAPE: 0.48255
LinearRegression Training Time: 0.03923 seconds
CPU Usage: 9.2 MHz
Memory Used: 8984.625 MB

Best parameters for SVR: {'C': 10, 'epsilon': 0.1}
SVR RMSE: 3.76407
SVR MAE: 2.94968
SVR R-squared: 0.98714
SVR SMAPE: 0.48638
SVR Training Time: 1.45788 seconds
CPU Usage: 37.3 MHz
Memory Used: 8992.79296875 MB

Best parameters for AdaBoostRegressor: {'learning_rate': 0.1, 'n_estimators': 200}
AdaBoostRegressor RMSE: 4.58039
AdaBoostRegressor MAE: 3.77324
AdaBoostRegressor R-squared: 0.98096
AdaBoostRegressor SMAPE: 0.49504
AdaBoostRegressor Training Time: 7.44464 seconds
CPU Usage: 23.5 MHz
Memory Used: 8989.75 MB

Best parameters for RandomForestRegressor: {'max_depth': 5, 'n_estimators': 50}
RandomForestRegressor RMSE: 3.98137
RandomForestRegressor MAE: 3.21772
RandomForestRegressor R-squared: 0.98561
RandomForestRegressor

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import hmean, gmean
import time
import psutil
import io
import requests
import tensorflow as tf

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1931 to 2008)
filtered_data = data[(data['Year'] >= 1931) & (data['Year'] <= 2008)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Normalize the target variable
# scaler = MinMaxScaler(feature_range=(0, 1))
# target_scaled = scaler.fit_transform(target)

# Split the data into training and testing sets
train_size = 840  # Number of months for training
test_size = 84  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

if X_test.shape[0] == 0:
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Define the regression models to be evaluated
models = [
    LinearRegression(),
    SVR(),
    AdaBoostRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor()
]

# Hyperparameter grids for the regression models
hyperparameter_grids = [
    {},
    {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.01, 0.001]},
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]},
    {'n_estimators': [50, 100, 200], 'max_depth': [None, 5, 10]},
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01, 0.001]}
]

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Train and evaluate each model with hyperparameter tuning
best_model_predictions = []
best_model_smapes = []
best_model_mses = []
best_model_rmses = []
best_model_maes = []
best_model_r2s = []
training_times = []
cpu_usages = []
memory_usages = []

for model, hyperparameters in zip(models, hyperparameter_grids):
    # Create a grid search instance
    grid_search = GridSearchCV(estimator=model, param_grid=hyperparameters, scoring='neg_mean_squared_error', cv=3, n_jobs=1)

    # Fit the grid search to the training data
    start_time = time.time()
    grid_search.fit(X_train, y_train.ravel())
    training_time = time.time() - start_time

    # Get the best model and its predictions
    best_model = grid_search.best_estimator_
    best_predictions = best_model.predict(X_test).ravel()

    # Append the best predictions to the list
    best_model_predictions.append(best_predictions)

    # Compute evaluation metrics for the best model
    mse = mean_squared_error(y_test, best_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, best_predictions)
    r2 = r2_score(y_test, best_predictions)
    smape_val = smape(y_test, best_predictions)

    best_model_smapes.append(smape_val)
    best_model_mses.append(mse)
    best_model_rmses.append(rmse)
    best_model_maes.append(mae)
    best_model_r2s.append(r2)

    # Print the best parameters and evaluation metrics
    model_name = model.__class__.__name__
    cpu_usage = psutil.cpu_percent()
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"{model_name} RMSE: {rmse:.5f}")
    print(f"{model_name} MAE: {mae:.5f}")
    print(f"{model_name} R-squared: {r2:.5f}")
    print(f"{model_name} SMAPE: {smape_val:.5f}")
    print(f"{model_name} Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {cpu_usage:.1f} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

    # Append training time, CPU usage, and memory usage to lists
    training_times.append(training_time)
    cpu_usages.append(cpu_usage)
    memory_usages.append(psutil.virtual_memory().used / 1024 / 1024)

# Combine the predictions of the best models
combined_predictions = np.mean(best_model_predictions, axis=0)
median_predictions = np.median(best_model_predictions, axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute harmonic mean and geometric mean of the predictions
harmonic_mean_predictions = hmean(best_model_predictions)
geometric_mean_predictions = gmean(best_model_predictions)

# Compute evaluation metrics for harmonic mean predictions
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for geometric mean predictions
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': [model.__class__.__name__ for model in models] + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': best_model_rmses + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': best_model_maes + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': best_model_r2s + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': best_model_smapes + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': training_times + [None] * 4,
    'CPU Usage (MHz)': cpu_usages + [None] * 4,
    'Memory Used (MB)': memory_usages + [None] * 4
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('Reg_eval_metrics_924_Months_1931_to_2008.csv', index=False)


Best parameters for LinearRegression: {}
LinearRegression RMSE: 3.26000
LinearRegression MAE: 2.71881
LinearRegression R-squared: 0.99690
LinearRegression SMAPE: 0.83592
LinearRegression Training Time: 0.03021 seconds
CPU Usage: 11.2 MHz
Memory Used: 9406.24609375 MB

Best parameters for SVR: {'C': 10, 'epsilon': 0.1}
SVR RMSE: 3.00136
SVR MAE: 2.45326
SVR R-squared: 0.99738
SVR SMAPE: 0.83531
SVR Training Time: 1.45736 seconds
CPU Usage: 28.0 MHz
Memory Used: 9417.50390625 MB

Best parameters for AdaBoostRegressor: {'learning_rate': 0.1, 'n_estimators': 200}
AdaBoostRegressor RMSE: 4.94923
AdaBoostRegressor MAE: 4.18209
AdaBoostRegressor R-squared: 0.99287
AdaBoostRegressor SMAPE: 0.82498
AdaBoostRegressor Training Time: 7.50766 seconds
CPU Usage: 21.0 MHz
Memory Used: 9417.0625 MB

Best parameters for RandomForestRegressor: {'max_depth': 10, 'n_estimators': 50}
RandomForestRegressor RMSE: 4.30944
RandomForestRegressor MAE: 3.10853
RandomForestRegressor R-squared: 0.99459
RandomForest

In [ ]:
# Create lists to store evaluation metrics
rmse_list = []
mae_list = []
r2_list = []
smape_list = []

for model_name, model in models.items():
    start_time = time.time()
    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                        y_train,
                        epochs=100,
                        batch_size=8,
                        verbose=0)
    training_time = time.time() - start_time

    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    smape_val = smape(y_test, predictions)

    # Store the predictions in the dictionary
    model_data[model_name] = predictions

    # Append evaluation metrics to the lists
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    smape_list.append(smape_val)

    print(f"Model: {model_name}")
    print(f"RMSE: {rmse:.5f}")
    print(f"MAE: {mae:.5f}")
    print(f"R-squared: {r2:.5f}")
    print(f"SMAPE: {smape_val:.5f}")
    print(f"Training Time: {training_time:.5f} seconds")
    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
    print()

# Combine the predictions of the models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Append the combined and median predictions to the model data dictionary
model_data['Combined Model'] = combined_predictions
model_data['Median Model'] = median_predictions

# Append the harmonic mean and geometric mean predictions to the model data dictionary
model_data['Harmonic Mean Model'] = harmonic_mean_predictions
model_data['Geometric Mean Model'] = geometric_mean_predictions

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL_forecasts_test3.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(models.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': rmse_list + [np.sqrt(mean_squared_error(y_test, combined_predictions)),
                         np.sqrt(mean_squared_error(y_test, median_predictions)),
                         np.sqrt(mean_squared_error(y_test, harmonic_mean_predictions)),
                         np.sqrt(mean_squared_error(y_test, geometric_mean_predictions))],
    'MAE': mae_list + [mean_absolute_error(y_test, combined_predictions),
                       mean_absolute_error(y_test, median_predictions),
                       mean_absolute_error(y_test, harmonic_mean_predictions),
                       mean_absolute_error(y_test, geometric_mean_predictions)],
    'R-squared': r2_list + [r2_score(y_test, combined_predictions),
                           r2_score(y_test, median_predictions),
                           r2_score(y_test, harmonic_mean_predictions),
                           r2_score(y_test, geometric_mean_predictions)],
    'SMAPE': smape_list + [smape(y_test, combined_predictions),
                           smape(y_test, median_predictions),
                           smape(y_test, harmonic_mean_predictions),
                           smape(y_test, geometric_mean_predictions)],
    'Training Time (seconds)': [training_time] * len(models) + [None, None, None, None],
    'CPU Usage (MHz)': [psutil.cpu_percent()] * len(models) + [None, None, None, None],
    'Memory Used (MB)': [psutil.virtual_memory().used / 1024 / 1024] * len(models) + [None, None, None, None],
    'Best Parameters': [None] * len(models) + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL_metrics_test3.csv', index=False)


In [ ]:
hyperparameters = {
    'fit_intercept': [True, False],  # Whether to include an intercept term
    'normalize': [True, False],  # Whether to normalize the input features
    'copy_X': [True, False],  # Whether to copy the input features or modify them in-place
    'n_jobs': [None, -1],  # Number of parallel jobs to run (-1 indicates using all available processors)
    'positive': [True, False],  # Whether to enforce positive coefficients
    'selection': ['cyclic', 'random'],  # Method for coefficient selection in Lasso (L1 regularization)
    'tol': [1e-4, 1e-5, 1e-6],  # Tolerance for stopping criteria
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],  # Solver algorithm for optimization
    'alpha_lasso': [0.1, 1.0, 10.0],  # Regularization strength (penalty term) for Lasso (L1) regularization
    'alpha_ridge': [0.1, 1.0, 10.0],  # Regularization strength (penalty term) for Ridge (L2) regularization
    'l1_ratio': [0.1, 0.5, 0.9]  # L1 ratio for Elastic Net regularization
}


# Deep learning 792 MONTHS (720 Months training, and 72 Testing) 
# Interval (1875-1941)

In [7]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [128, 15], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [128, 15], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [128, 15], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [128, 15], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None

    for units in params['units']:
        for learning_rate in params['learning_rate']:
            for batch_size in params['batch_size']:
                if model_name == 'LSTM':
                    model = create_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'Stacked LSTM':
                    model = create_stacked_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'Bidirectional LSTM':
                    model = create_bidirectional_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'GRU':
                    model = create_gru_model(units, learning_rate, batch_size)

                start_time = time.time()
                history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                    y_train,
                                    epochs=100,
                                    batch_size=batch_size,
                                    verbose=0)
                training_time = time.time() - start_time

                predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                mse = mean_squared_error(y_test, predictions)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(y_test, predictions)
                r2 = r2_score(y_test, predictions)
                smape_val = smape(y_test, predictions)

                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = model

                print(f"Model: {model_name} (Units: {units}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                print(f"RMSE: {rmse:.5f}")
                print(f"MAE: {mae:.5f}")
                print(f"R-squared: {r2:.5f}")
                print(f"SMAPE: {smape_val:.5f}")
                print(f"Training Time: {training_time:.5f} seconds")
                print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics in the eval metrics dictionary
    eval_metrics[model_name] = {
        'RMSE': best_rmse,
        'MAE': mae,
        'R-squared': r2,
        'SMAPE': smape_val,
        'Training Time (seconds)': training_time,
        'CPU Usage (MHz)': psutil.cpu_percent(),
        'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL_forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL_eval_metrics_792_Months_1875_to_1941.csv', index=False)


3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 128, Learning Rate: 0.005, Batch Size: 8)
RMSE: 18.70207
MAE: 14.82553
R-squared: 0.81603
SMAPE: 0.39547
Training Time: 19.95766 seconds
CPU Usage: 18.5 MHz
Memory Used: 13691.76953125 MB

3/3 [==============================] - 1s 3ms/step
Model: LSTM (Units: 15, Learning Rate: 0.005, Batch Size: 8)
RMSE: 66.45579
MAE: 56.53208
R-squared: -1.32293
SMAPE: 0.55470
Training Time: 18.88808 seconds
CPU Usage: 24.6 MHz
Memory Used: 13736.640625 MB

3/3 [==============================] - 1s 3ms/step
Model: Stacked LSTM (Units: 128, Learning Rate: 0.005, Batch Size: 8)
RMSE: 17.07815
MAE: 13.25761
R-squared: 0.84659
SMAPE: 0.39511
Training Time: 48.08800 seconds
CPU Usage: 36.5 MHz
Memory Used: 13744.66796875 MB

3/3 [==============================] - 1s 2ms/step
Model: Stacked LSTM (Units: 15, Learning Rate: 0.005, Batch Size: 8)
RMSE: 65.44545
MAE: 55.13991
R-squared: -1.25283
SMAPE: 0.54238
Training Time: 29.88610 second

In [9]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [32, 64, 128], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [32, 64, 128], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [32, 64, 128], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [32, 64, 128], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_params = {}

    for units in params['units']:
        for learning_rate in params['learning_rate']:
            for batch_size in params['batch_size']:
                if model_name == 'LSTM':
                    model = create_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'Stacked LSTM':
                    model = create_stacked_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'Bidirectional LSTM':
                    model = create_bidirectional_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'GRU':
                    model = create_gru_model(units, learning_rate, batch_size)

                start_time = time.time()
                history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                    y_train,
                                    epochs=100,
                                    batch_size=batch_size,
                                    verbose=0)
                training_time = time.time() - start_time

                predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                mse = mean_squared_error(y_test, predictions)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(y_test, predictions)
                r2 = r2_score(y_test, predictions)
                smape_val = smape(y_test, predictions)

                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = model
                    best_params = {
                        'Units': units,
                        'Learning Rate': learning_rate,
                        'Batch Size': batch_size
                    }

                print(f"Model: {model_name} (Units: {units}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                print(f"RMSE: {rmse:.5f}")
                print(f"MAE: {mae:.5f}")
                print(f"R-squared: {r2:.5f}")
                print(f"SMAPE: {smape_val:.5f}")
                print(f"Training Time: {training_time:.5f} seconds")
                print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics and best parameters in the eval metrics dictionary
    eval_metrics[model_name] = {
        'Best RMSE': best_rmse,
        'Best MAE': mae,
        'Best R-squared': r2,
        'Best SMAPE': smape_val,
        'Best Training Time (seconds)': training_time,
        'Best CPU Usage (MHz)': psutil.cpu_percent(),
        'Best Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024,
        'Best Parameters': best_params
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL_forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['Best RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['Best MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['Best R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['Best SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Best Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['Best CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Best Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Best Parameters': [eval_metrics[model]['Best Parameters'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL5_eval_metrics_792_Months_1875_to_1941.csv', index=False)


3/3 [==============================] - 1s 1ms/step
Model: LSTM (Units: 32, Learning Rate: 0.001, Batch Size: 8)
RMSE: 38.03047
MAE: 29.01166
R-squared: 0.23927
SMAPE: 0.39805
Training Time: 17.20945 seconds
CPU Usage: 19.3 MHz
Memory Used: 15803.76171875 MB

3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 64, Learning Rate: 0.001, Batch Size: 8)
RMSE: 24.43683
MAE: 18.72685
R-squared: 0.68591
SMAPE: 0.39224
Training Time: 18.74232 seconds
CPU Usage: 23.6 MHz
Memory Used: 15834.109375 MB

3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 128, Learning Rate: 0.001, Batch Size: 8)
RMSE: 16.22849
MAE: 12.46412
R-squared: 0.86148
SMAPE: 0.38390
Training Time: 23.61059 seconds
CPU Usage: 29.8 MHz
Memory Used: 15873.1875 MB

3/3 [==============================] - 1s 2ms/step
Model: Stacked LSTM (Units: 32, Learning Rate: 0.001, Batch Size: 8)
RMSE: 38.60747
MAE: 29.82029
R-squared: 0.21601
SMAPE: 0.40810
Training Time: 26.78897 seconds
CPU Usage: 29

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2, 0.4, 0.6, 0.8], 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3], 'batch_size': [8, 16, 32, 64, 128]},
    'Stacked LSTM': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2, 0.4, 0.6, 0.8], 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3], 'batch_size': [8, 16, 32, 64, 128]},
    'Bidirectional LSTM': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2, 0.4, 0.6, 0.8], 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3], 'batch_size': [8, 16, 32, 64, 128]},
    'GRU': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2, 0.4, 0.6, 0.8], 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3], 'batch_size': [8, 16, 32, 64, 128]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_params = {}

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            for learning_rate in params['learning_rate']:
                for batch_size in params['batch_size']:
                    if model_name == 'LSTM':
                        model = create_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Stacked LSTM':
                        model = create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Bidirectional LSTM':
                        model = create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'GRU':
                        model = create_gru_model(units, dropout_rate, learning_rate, batch_size)

                    start_time = time.time()
                    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                        y_train,
                                        epochs=100,
                                        batch_size=batch_size,
                                        verbose=0)
                    training_time = time.time() - start_time

                    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                    mse = mean_squared_error(y_test, predictions)
                    rmse = np.sqrt(mse)
                    mae = mean_absolute_error(y_test, predictions)
                    r2 = r2_score(y_test, predictions)
                    smape_val = smape(y_test, predictions)

                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_model = model
                        best_params = {
                            'Units': units,
                            'Dropout Rate': dropout_rate,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size
                        }

                    print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                    print(f"RMSE: {rmse:.5f}")
                    print(f"MAE: {mae:.5f}")
                    print(f"R-squared: {r2:.5f}")
                    print(f"SMAPE: {smape_val:.5f}")
                    print(f"Training Time: {training_time:.5f} seconds")
                    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                    print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics and best parameters in the eval metrics dictionary
    eval_metrics[model_name] = {
        'Best RMSE': best_rmse,
        'Best MAE': mae,
        'Best R-squared': r2,
        'Best SMAPE': smape_val,
        'Best Training Time (seconds)': training_time,
        'Best CPU Usage (MHz)': psutil.cpu_percent(),
        'Best Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024,
        'Best Parameters': best_params
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL_forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['Best RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['Best MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['Best R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['Best SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Best Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['Best CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Best Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Best Parameters': [eval_metrics[model]['Best Parameters'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL6_eval_metrics_792_Months_1875_to_1941.csv', index=False)


# Reported DL

In [10]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [8, 32, 64, 128, 256], 'dropout_rate': [0.0, 0.2], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_params = {}

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            for learning_rate in params['learning_rate']:
                for batch_size in params['batch_size']:
                    if model_name == 'LSTM':
                        model = create_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Stacked LSTM':
                        model = create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Bidirectional LSTM':
                        model = create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'GRU':
                        model = create_gru_model(units, dropout_rate, learning_rate, batch_size)

                    start_time = time.time()
                    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                        y_train,
                                        epochs=100,
                                        batch_size=batch_size,
                                        verbose=0)
                    training_time = time.time() - start_time

                    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                    mse = mean_squared_error(y_test, predictions)
                    rmse = np.sqrt(mse)
                    mae = mean_absolute_error(y_test, predictions)
                    r2 = r2_score(y_test, predictions)
                    smape_val = smape(y_test, predictions)

                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_model = model
                        best_params = {
                            'Units': units,
                            'Dropout Rate': dropout_rate,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size
                        }

                    print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                    print(f"RMSE: {rmse:.5f}")
                    print(f"MAE: {mae:.5f}")
                    print(f"R-squared: {r2:.5f}")
                    print(f"SMAPE: {smape_val:.5f}")
                    print(f"Training Time: {training_time:.5f} seconds")
                    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                    print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics and best parameters in the eval metrics dictionary
    eval_metrics[model_name] = {
        'Best RMSE': best_rmse,
        'Best MAE': mae,
        'Best R-squared': r2,
        'Best SMAPE': smape_val,
        'Best Training Time (seconds)': training_time,
        'Best CPU Usage (MHz)': psutil.cpu_percent(),
        'Best Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024,
        'Best Parameters': best_params
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL_forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['Best RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['Best MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['Best R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['Best SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Best Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['Best CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Best Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Best Parameters': [eval_metrics[model]['Best Parameters'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL6_eval_metrics_792_Months_1875_to_1941.csv', index=False)


3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 8, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 91.42544
MAE: 81.81916
R-squared: -3.39647
SMAPE: 0.80417
Training Time: 16.85487 seconds
CPU Usage: 27.2 MHz
Memory Used: 16181.10546875 MB

3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 8, Dropout Rate: 0.2, Learning Rate: 0.001, Batch Size: 8)
RMSE: 92.10337
MAE: 82.50260
R-squared: -3.46191
SMAPE: 0.81196
Training Time: 18.19880 seconds
CPU Usage: 26.7 MHz
Memory Used: 16176.203125 MB

3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 32, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 38.26613
MAE: 29.41134
R-squared: 0.22981
SMAPE: 0.40468
Training Time: 17.45640 seconds
CPU Usage: 24.1 MHz
Memory Used: 16154.16796875 MB

3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 32, Dropout Rate: 0.2, Learning Rate: 0.001, Batch Size: 8)
RMSE: 38.95418
MAE: 30.30908
R-square

3/3 [==============================] - 1s 3ms/step
Model: Bidirectional LSTM (Units: 256, Dropout Rate: 0.2, Learning Rate: 0.001, Batch Size: 8)
RMSE: 10.57062
MAE: 8.70269
R-squared: 0.94123
SMAPE: 0.39292
Training Time: 85.88237 seconds
CPU Usage: 70.0 MHz
Memory Used: 16586.6171875 MB

3/3 [==============================] - 0s 2ms/step
Model: GRU (Units: 8, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 82.98619
MAE: 73.17269
R-squared: -2.62227
SMAPE: 0.70731
Training Time: 19.60758 seconds
CPU Usage: 26.6 MHz
Memory Used: 16652.9296875 MB

3/3 [==============================] - 0s 2ms/step
Model: GRU (Units: 8, Dropout Rate: 0.2, Learning Rate: 0.001, Batch Size: 8)
RMSE: 82.20990
MAE: 72.11019
R-squared: -2.55482
SMAPE: 0.69453
Training Time: 19.94442 seconds
CPU Usage: 27.8 MHz
Memory Used: 16704.84765625 MB

3/3 [==============================] - 0s 2ms/step
Model: GRU (Units: 32, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 28.52612
MAE: 21.197

In [13]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [128, 256], 'dropout_rate': [0.0,], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_params = {}

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            for learning_rate in params['learning_rate']:
                for batch_size in params['batch_size']:
                    if model_name == 'LSTM':
                        model = create_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Stacked LSTM':
                        model = create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Bidirectional LSTM':
                        model = create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'GRU':
                        model = create_gru_model(units, dropout_rate, learning_rate, batch_size)

                    start_time = time.time()
                    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                        y_train,
                                        epochs=100,
                                        batch_size=batch_size,
                                        verbose=0)
                    training_time = time.time() - start_time

                    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                    mse = mean_squared_error(y_test, predictions)
                    rmse = np.sqrt(mse)
                    mae = mean_absolute_error(y_test, predictions)
                    r2 = r2_score(y_test, predictions)
                    smape_val = smape(y_test, predictions)

                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_model = model
                        best_params = {
                            'Units': units,
                            'Dropout Rate': dropout_rate,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size
                        }

                    print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                    print(f"RMSE: {rmse:.5f}")
                    print(f"MAE: {mae:.5f}")
                    print(f"R-squared: {r2:.5f}")
                    print(f"SMAPE: {smape_val:.5f}")
                    print(f"Training Time: {training_time:.5f} seconds")
                    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                    print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics and best parameters in the eval metrics dictionary
    eval_metrics[model_name] = {
        'Best RMSE': best_rmse,
        'Best MAE': mae,
        'Best R-squared': r2,
        'Best SMAPE': smape_val,
        'Best Training Time (seconds)': training_time,
        'Best CPU Usage (MHz)': psutil.cpu_percent(),
        'Best Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024,
        'Best Parameters': best_params
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL7_forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['Best RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['Best MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['Best R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['Best SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Best Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['Best CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Best Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Best Parameters': [eval_metrics[model]['Best Parameters'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL7_eval_metrics_792_Months_1875_to_1941.csv', index=False)


3/3 [==============================] - 1s 3ms/step
Model: LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 14.70907
MAE: 11.33125
R-squared: 0.88620
SMAPE: 0.38716
Training Time: 23.83243 seconds
CPU Usage: 16.0 MHz
Memory Used: 16752.44140625 MB

3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 10.66707
MAE: 8.65575
R-squared: 0.94015
SMAPE: 0.39201
Training Time: 64.89703 seconds
CPU Usage: 55.7 MHz
Memory Used: 16742.14453125 MB

3/3 [==============================] - 1s 3ms/step
Model: Stacked LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 13.53325
MAE: 10.37206
R-squared: 0.90367
SMAPE: 0.39146
Training Time: 42.28091 seconds
CPU Usage: 32.6 MHz
Memory Used: 16913.3359375 MB

3/3 [==============================] - 1s 3ms/step
Model: Stacked LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 14.94966
MA

In [14]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1845 to 1931)
filtered_data = data[(data['Year'] >= 1845) & (data['Year'] <= 1931)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 840  # Number of months for training
test_size = 84  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [128, 256], 'dropout_rate': [0.0,], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_params = {}

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            for learning_rate in params['learning_rate']:
                for batch_size in params['batch_size']:
                    if model_name == 'LSTM':
                        model = create_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Stacked LSTM':
                        model = create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Bidirectional LSTM':
                        model = create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'GRU':
                        model = create_gru_model(units, dropout_rate, learning_rate, batch_size)

                    start_time = time.time()
                    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                        y_train,
                                        epochs=100,
                                        batch_size=batch_size,
                                        verbose=0)
                    training_time = time.time() - start_time

                    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                    mse = mean_squared_error(y_test, predictions)
                    rmse = np.sqrt(mse)
                    mae = mean_absolute_error(y_test, predictions)
                    r2 = r2_score(y_test, predictions)
                    smape_val = smape(y_test, predictions)

                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_model = model
                        best_params = {
                            'Units': units,
                            'Dropout Rate': dropout_rate,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size
                        }

                    print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                    print(f"RMSE: {rmse:.5f}")
                    print(f"MAE: {mae:.5f}")
                    print(f"R-squared: {r2:.5f}")
                    print(f"SMAPE: {smape_val:.5f}")
                    print(f"Training Time: {training_time:.5f} seconds")
                    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                    print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics and best parameters in the eval metrics dictionary
    eval_metrics[model_name] = {
        'Best RMSE': best_rmse,
        'Best MAE': mae,
        'Best R-squared': r2,
        'Best SMAPE': smape_val,
        'Best Training Time (seconds)': training_time,
        'Best CPU Usage (MHz)': psutil.cpu_percent(),
        'Best Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024,
        'Best Parameters': best_params
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL8_forecasts_924_Months_840tr_84t_1854_to_1931.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['Best RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['Best MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['Best R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['Best SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Best Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['Best CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Best Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Best Parameters': [eval_metrics[model]['Best Parameters'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL8_eval_metrics_924_Months_1854_to_1931.csv', index=False)


3/3 [==============================] - 0s 2ms/step
Model: LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 4.77441
MAE: 3.73505
R-squared: 0.98550
SMAPE: 0.47405
Training Time: 26.25036 seconds
CPU Usage: 12.4 MHz
Memory Used: 16325.8984375 MB

3/3 [==============================] - 0s 3ms/step
Model: LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 4.77488
MAE: 3.70249
R-squared: 0.98550
SMAPE: 0.46071
Training Time: 75.59181 seconds
CPU Usage: 50.1 MHz
Memory Used: 16313.5234375 MB

3/3 [==============================] - 1s 3ms/step
Model: Stacked LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 5.64113
MAE: 4.17612
R-squared: 0.97976
SMAPE: 0.45797
Training Time: 53.69549 seconds
CPU Usage: 36.9 MHz
Memory Used: 15733.30078125 MB

3/3 [==============================] - 1s 4ms/step
Model: Stacked LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 4.68559
MAE: 3.53

In [15]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1931 to 2008)
filtered_data = data[(data['Year'] >= 1931) & (data['Year'] <= 2008)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 840  # Number of months for training
test_size = 84  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [128, 256], 'dropout_rate': [0.0,], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_params = {}

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            for learning_rate in params['learning_rate']:
                for batch_size in params['batch_size']:
                    if model_name == 'LSTM':
                        model = create_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Stacked LSTM':
                        model = create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Bidirectional LSTM':
                        model = create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'GRU':
                        model = create_gru_model(units, dropout_rate, learning_rate, batch_size)

                    start_time = time.time()
                    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                        y_train,
                                        epochs=100,
                                        batch_size=batch_size,
                                        verbose=0)
                    training_time = time.time() - start_time

                    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                    mse = mean_squared_error(y_test, predictions)
                    rmse = np.sqrt(mse)
                    mae = mean_absolute_error(y_test, predictions)
                    r2 = r2_score(y_test, predictions)
                    smape_val = smape(y_test, predictions)

                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_model = model
                        best_params = {
                            'Units': units,
                            'Dropout Rate': dropout_rate,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size
                        }

                    print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                    print(f"RMSE: {rmse:.5f}")
                    print(f"MAE: {mae:.5f}")
                    print(f"R-squared: {r2:.5f}")
                    print(f"SMAPE: {smape_val:.5f}")
                    print(f"Training Time: {training_time:.5f} seconds")
                    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                    print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics and best parameters in the eval metrics dictionary
    eval_metrics[model_name] = {
        'Best RMSE': best_rmse,
        'Best MAE': mae,
        'Best R-squared': r2,
        'Best SMAPE': smape_val,
        'Best Training Time (seconds)': training_time,
        'Best CPU Usage (MHz)': psutil.cpu_percent(),
        'Best Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024,
        'Best Parameters': best_params
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL9_forecasts_924_Months_840tr_84t_1931_to_2008.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['Best RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['Best MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['Best R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['Best SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Best Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['Best CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Best Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Best Parameters': [eval_metrics[model]['Best Parameters'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL9_eval_metrics_924_Months_1931_to_2008.csv', index=False)


3/3 [==============================] - 0s 3ms/step
Model: LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 4.67070
MAE: 4.31634
R-squared: 0.99365
SMAPE: 0.82408
Training Time: 26.24233 seconds
CPU Usage: 15.5 MHz
Memory Used: 16002.421875 MB

3/3 [==============================] - 0s 3ms/step
Model: LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 2.82235
MAE: 2.25386
R-squared: 0.99768
SMAPE: 0.83699
Training Time: 79.33321 seconds
CPU Usage: 49.6 MHz
Memory Used: 15930.3984375 MB

3/3 [==============================] - 1s 3ms/step
Model: Stacked LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 4.11088
MAE: 3.41872
R-squared: 0.99508
SMAPE: 0.83544
Training Time: 48.95841 seconds
CPU Usage: 31.8 MHz
Memory Used: 15988.08984375 MB

3/3 [==============================] - 1s 4ms/step
Model: Stacked LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 3.63895
MAE: 2.860

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import hmean, gmean
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [128, 256], 'dropout_rate': [0.0], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [128, 256], 'dropout_rate': [0.0,], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_params = {}

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            for learning_rate in params['learning_rate']:
                for batch_size in params['batch_size']:
                    if model_name == 'LSTM':
                        model = create_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Stacked LSTM':
                        model = create_stacked_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'Bidirectional LSTM':
                        model = create_bidirectional_lstm_model(units, dropout_rate, learning_rate, batch_size)
                    elif model_name == 'GRU':
                        model = create_gru_model(units, dropout_rate, learning_rate, batch_size)

                    start_time = time.time()
                    history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                        y_train,
                                        epochs=100,
                                        batch_size=batch_size,
                                        verbose=0)
                    training_time = time.time() - start_time

                    predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                    mse = mean_squared_error(y_test, predictions)
                    rmse = np.sqrt(mse)
                    mae = mean_absolute_error(y_test, predictions)
                    r2 = r2_score(y_test, predictions)
                    smape_val = smape(y_test, predictions)

                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_model = model
                        best_params = {
                            'Units': units,
                            'Dropout Rate': dropout_rate,
                            'Learning Rate': learning_rate,
                            'Batch Size': batch_size
                        }

                    print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                    print(f"RMSE: {rmse:.5f}")
                    print(f"MAE: {mae:.5f}")
                    print(f"R-squared: {r2:.5f}")
                    print(f"SMAPE: {smape_val:.5f}")
                    print(f"Training Time: {training_time:.5f} seconds")
                    print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                    print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                    print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics and best parameters in the eval metrics dictionary
    eval_metrics[model_name] = {
        'Best RMSE': best_rmse,
        'Best MAE': mae,
        'Best R-squared': r2,
        'Best SMAPE': smape_val,
        'Best Training Time (seconds)': training_time,
        'Best CPU Usage (MHz)': psutil.cpu_percent(),
        'Best Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024,
        'Best Parameters': best_params
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)
harmonic_mean_predictions = hmean(list(model_data.values()))
geometric_mean_predictions = gmean(list(model_data.values()))

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Compute evaluation metrics for the harmonic mean model
harmonic_mean_mse = mean_squared_error(y_test, harmonic_mean_predictions)
harmonic_mean_rmse = np.sqrt(harmonic_mean_mse)
harmonic_mean_mae = mean_absolute_error(y_test, harmonic_mean_predictions)
harmonic_mean_r2 = r2_score(y_test, harmonic_mean_predictions)
harmonic_mean_smape = smape(y_test, harmonic_mean_predictions)

# Compute evaluation metrics for the geometric mean model
geometric_mean_mse = mean_squared_error(y_test, geometric_mean_predictions)
geometric_mean_rmse = np.sqrt(geometric_mean_mse)
geometric_mean_mae = mean_absolute_error(y_test, geometric_mean_predictions)
geometric_mean_r2 = r2_score(y_test, geometric_mean_predictions)
geometric_mean_smape = smape(y_test, geometric_mean_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('DL10_forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Prepare the data for the evaluation metrics table
metrics_data = {
    'Model': list(hyperparameters.keys()) + ['Combined Model', 'Median Model', 'Harmonic Mean Model', 'Geometric Mean Model'],
    'RMSE': [eval_metrics[model]['Best RMSE'] for model in hyperparameters.keys()] + [combined_rmse, median_rmse, harmonic_mean_rmse, geometric_mean_rmse],
    'MAE': [eval_metrics[model]['Best MAE'] for model in hyperparameters.keys()] + [combined_mae, median_mae, harmonic_mean_mae, geometric_mean_mae],
    'R-squared': [eval_metrics[model]['Best R-squared'] for model in hyperparameters.keys()] + [combined_r2, median_r2, harmonic_mean_r2, geometric_mean_r2],
    'SMAPE': [eval_metrics[model]['Best SMAPE'] for model in hyperparameters.keys()] + [combined_smape, median_smape, harmonic_mean_smape, geometric_mean_smape],
    'Training Time (seconds)': [eval_metrics[model]['Best Training Time (seconds)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'CPU Usage (MHz)': [eval_metrics[model]['Best CPU Usage (MHz)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Memory Used (MB)': [eval_metrics[model]['Best Memory Used (MB)'] for model in hyperparameters.keys()] + [None, None, None, None],
    'Best Parameters': [eval_metrics[model]['Best Parameters'] for model in hyperparameters.keys()] + [None, None, None, None]
}

# Save the evaluation metrics to a CSV file
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv('DL10_eval_metrics_792_Months_720tr_72t_1875_to_1941.csv', index=False)


3/3 [==============================] - 1s 4ms/step
Model: LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 14.51358
MAE: 11.34709
R-squared: 0.88921
SMAPE: 0.39203
Training Time: 42.55742 seconds
CPU Usage: 43.6 MHz
Memory Used: 8263.48046875 MB

3/3 [==============================] - 1s 3ms/step
Model: LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 10.83939
MAE: 8.85193
R-squared: 0.93820
SMAPE: 0.39453
Training Time: 75.23962 seconds
CPU Usage: 69.8 MHz
Memory Used: 8363.94140625 MB

3/3 [==============================] - 1s 4ms/step
Model: Stacked LSTM (Units: 128, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 13.08240
MAE: 9.86323
R-squared: 0.90998
SMAPE: 0.38343
Training Time: 44.58561 seconds
CPU Usage: 47.2 MHz
Memory Used: 8396.31640625 MB

3/3 [==============================] - 1s 9ms/step
Model: Stacked LSTM (Units: 256, Dropout Rate: 0.0, Learning Rate: 0.001, Batch Size: 8)
RMSE: 13.03676
MAE: 

In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, learning_rate, batch_size):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [128], 'learning_rate': [0.001], 'batch_size': [8]},
    'Stacked LSTM': {'units': [128], 'learning_rate': [0.001], 'batch_size': [8]},
    'Bidirectional LSTM': {'units': [128], 'learning_rate': [0.001], 'batch_size': [8]},
    'GRU': {'units': [128], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None

    for units in params['units']:
        for learning_rate in params['learning_rate']:
            for batch_size in params['batch_size']:
                if model_name == 'LSTM':
                    model = create_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'Stacked LSTM':
                    model = create_stacked_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'Bidirectional LSTM':
                    model = create_bidirectional_lstm_model(units, learning_rate, batch_size)
                elif model_name == 'GRU':
                    model = create_gru_model(units, learning_rate, batch_size)

                start_time = time.time()
                history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                    y_train,
                                    epochs=100,
                                    batch_size=batch_size,
                                    verbose=0)
                training_time = time.time() - start_time

                predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
                mse = mean_squared_error(y_test, predictions)
                rmse = np.sqrt(mse)
                mae = mean_absolute_error(y_test, predictions)
                r2 = r2_score(y_test, predictions)
                smape_val = smape(y_test, predictions)

                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = model

                print(f"Model: {model_name} (Units: {units}, Learning Rate: {learning_rate}, Batch Size: {batch_size})")
                print(f"RMSE: {rmse:.5f}")
                print(f"MAE: {mae:.5f}")
                print(f"R-squared: {r2:.5f}")
                print(f"SMAPE: {smape_val:.5f}")
                print(f"Training Time: {training_time:.5f} seconds")
                print(f"CPU Usage: {psutil.cpu_percent()} MHz")
                print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
                print()

    # Store the predictions in the dictionary
    model_data[model_name] = best_model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics in the eval metrics dictionary
    eval_metrics[model_name] = {
        'RMSE': best_rmse,
        'MAE': mae,
        'R-squared': r2,
        'SMAPE': smape_val,
        'Training Time (seconds)': training_time,
        'CPU Usage (MHz)': psutil.cpu_percent(),
        'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
    }

# Combine the predictions of the best models
combined_predictions = np.mean(list(model_data.values()), axis=0)
median_predictions = np.median(list(model_data.values()), axis=0)

# Compute evaluation metrics for the combined model
combined_mse = mean_squared_error(y_test, combined_predictions)
combined_rmse = np.sqrt(combined_mse)
combined_mae = mean_absolute_error(y_test, combined_predictions)
combined_r2 = r2_score(y_test, combined_predictions)
combined_smape = smape(y_test, combined_predictions)

# Compute evaluation metrics for the median model
median_mse = mean_squared_error(y_test, median_predictions)
median_rmse = np.sqrt(median_mse)
median_mae = mean_absolute_error(y_test, median_predictions)
median_r2 = r2_score(y_test, median_predictions)
median_smape = smape(y_test, median_predictions)

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Save evaluation metrics to a CSV file
df_metrics = pd.DataFrame(eval_metrics).T
df_metrics.to_csv('DL_eval_metrics_792_Months_1875_to_1941.csv')

# Print true values and forecasts in table format
df_true_values = pd.DataFrame(y_test, columns=['True Value'])
df_forecasts = pd.DataFrame(model_data)

print("True Values and Forecasts:")
print(pd.concat([df_true_values, df_forecasts], axis=1))


3/3 [==============================] - 0s 3ms/step
Model: LSTM (Units: 128, Learning Rate: 0.001, Batch Size: 8)
RMSE: 16.76721
MAE: 13.12013
R-squared: 0.85213
SMAPE: 0.39016
Training Time: 26.84632 seconds
CPU Usage: 25.3 MHz
Memory Used: 13020.5 MB

3/3 [==============================] - 1s 3ms/step
Model: Stacked LSTM (Units: 128, Learning Rate: 0.001, Batch Size: 8)
RMSE: 15.51223
MAE: 12.08561
R-squared: 0.87343
SMAPE: 0.37858
Training Time: 52.40342 seconds
CPU Usage: 34.8 MHz
Memory Used: 13056.6640625 MB

3/3 [==============================] - 1s 4ms/step
Model: Bidirectional LSTM (Units: 128, Learning Rate: 0.001, Batch Size: 8)
RMSE: 15.57322
MAE: 12.59646
R-squared: 0.87244
SMAPE: 0.39772
Training Time: 38.10903 seconds
CPU Usage: 34.7 MHz
Memory Used: 13126.8046875 MB

3/3 [==============================] - 0s 3ms/step
Model: GRU (Units: 128, Learning Rate: 0.001, Batch Size: 8)
RMSE: 14.88445
MAE: 11.42647
R-squared: 0.88347
SMAPE: 0.38509
Training Time: 27.87644 seconds


In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'Stacked LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'Bidirectional LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'GRU': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_units = 0
    best_dropout_rate = 0

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            if model_name == 'LSTM':
                model = create_lstm_model(units, dropout_rate)
            elif model_name == 'Stacked LSTM':
                model = create_stacked_lstm_model(units, dropout_rate)
            elif model_name == 'Bidirectional LSTM':
                model = create_bidirectional_lstm_model(units, dropout_rate)
            elif model_name == 'GRU':
                model = create_gru_model(units, dropout_rate)

            start_time = time.time()
            history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                y_train,
                                epochs=100,
                                batch_size=32,
                                verbose=0)
            training_time = time.time() - start_time

            predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, predictions)
            r2 = r2_score(y_test, predictions)
            smape_val = smape(y_test, predictions)

            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
                best_units = units
                best_dropout_rate = dropout_rate

            print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate})")
            print(f"RMSE: {rmse:.5f}")
            print(f"MAE: {mae:.5f}")
            print(f"R-squared: {r2:.5f}")
            print(f"SMAPE: {smape_val:.5f}")
            print(f"Training Time: {training_time:.5f} seconds")
            print(f"CPU Usage: {psutil.cpu_percent()} MHz")
            print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
            print()

    # Store the predictions in the dictionary
    model_data[f"{model_name} (Units: {best_units}, Dropout Rate: {best_dropout_rate})"] = best_model.predict(
        X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics in the eval metrics dictionary
    eval_metrics[model_name] = {
        'RMSE': best_rmse,
        'MAE': mae,
        'R-squared': r2,
        'SMAPE': smape_val,
        'Training Time (seconds)': training_time,
        'CPU Usage (MHz)': psutil.cpu_percent(),
        'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
    }

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Save evaluation metrics to a CSV file
df_metrics = pd.DataFrame(eval_metrics).T
df_metrics.to_csv('DL_eval_metrics_792_Months_1875_to_1941.csv')

# Print true values and forecasts in table format
df_true_values = pd.DataFrame(y_test, columns=['True Value'])
df_forecasts = pd.DataFrame(model_data)

print("True Values and Forecasts:")
print(pd.concat([df_true_values, df_forecasts], axis=1))


3/3 [==============================] - 1s 8ms/step
Model: LSTM (Units: 32, Dropout Rate: 0.0)
RMSE: 97.99311
MAE: 88.53317
R-squared: -4.05081
SMAPE: 0.89003
Training Time: 12.47534 seconds
CPU Usage: 34.6 MHz
Memory Used: 14601.671875 MB

3/3 [==============================] - 1s 2ms/step
Model: LSTM (Units: 32, Dropout Rate: 0.2)
RMSE: 97.43258
MAE: 87.97609
R-squared: -3.99319
SMAPE: 0.88257
Training Time: 12.35956 seconds
CPU Usage: 41.6 MHz
Memory Used: 14647.19921875 MB

3/3 [==============================] - 1s 8ms/step
Model: LSTM (Units: 32, Dropout Rate: 0.4)
RMSE: 95.26366
MAE: 85.71160
R-squared: -3.77336
SMAPE: 0.85180
Training Time: 12.30328 seconds
CPU Usage: 45.4 MHz
Memory Used: 14718.7734375 MB

3/3 [==============================] - 1s 8ms/step
Model: LSTM (Units: 64, Dropout Rate: 0.0)
RMSE: 66.57993
MAE: 56.11668
R-squared: -1.33161
SMAPE: 0.54892
Training Time: 12.75124 seconds
CPU Usage: 43.8 MHz
Memory Used: 14721.24609375 MB

3/3 [==============================

3/3 [==============================] - 1s 6ms/step
Model: GRU (Units: 32, Dropout Rate: 0.0)
RMSE: 85.15697
MAE: 75.28430
R-squared: -2.81426
SMAPE: 0.72890
Training Time: 12.40072 seconds
CPU Usage: 44.7 MHz
Memory Used: 15557.17578125 MB

3/3 [==============================] - 1s 8ms/step
Model: GRU (Units: 32, Dropout Rate: 0.2)
RMSE: 83.15985
MAE: 73.19826
R-squared: -2.63745
SMAPE: 0.70633
Training Time: 12.58051 seconds
CPU Usage: 33.3 MHz
Memory Used: 15575.30859375 MB

3/3 [==============================] - 1s 5ms/step
Model: GRU (Units: 32, Dropout Rate: 0.4)
RMSE: 86.92422
MAE: 77.14907
R-squared: -2.97421
SMAPE: 0.74901
Training Time: 13.60795 seconds
CPU Usage: 36.8 MHz
Memory Used: 15589.1875 MB

3/3 [==============================] - 1s 8ms/step
Model: GRU (Units: 64, Dropout Rate: 0.0)
RMSE: 60.73946
MAE: 49.98301
R-squared: -0.94049
SMAPE: 0.50175
Training Time: 13.36260 seconds
CPU Usage: 32.8 MHz
Memory Used: 15594.65234375 MB

3/3 [==============================] - 1

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests
import tensorflow as tf

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the LSTM model
def create_lstm_model(units, learning_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# Hyperparameters for the LSTM model
hyperparameters = {
    'LSTM': {'units': [128], 'learning_rate': [0.001], 'batch_size': [8]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each LSTM model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_units = 0
    best_learning_rate = 0

    for units in params['units']:
        for learning_rate in params['learning_rate']:
            model = create_lstm_model(units, learning_rate)

            start_time = time.time()
            history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                y_train,
                                epochs=100,
                                batch_size=params['batch_size'][0],
                                verbose=0)
            training_time = time.time() - start_time

            predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, predictions)
            r2 = r2_score(y_test, predictions)
            smape_val = smape(y_test, predictions)

            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
                best_units = units
                best_learning_rate = learning_rate

            print(f"Model: {model_name} (Units: {units}, Learning Rate: {learning_rate})")
            print(f"RMSE: {rmse:.5f}")
            print(f"MAE: {mae:.5f}")
            print(f"R-squared: {r2:.5f}")
            print(f"SMAPE: {smape_val:.5f}")
            print(f"Training Time: {training_time:.5f} seconds")
            print(f"CPU Usage: {psutil.cpu_percent()} MHz")
            print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
            print()

    # Store the predictions in the dictionary
    model_data[f"{model_name} (Units: {best_units}, Learning Rate: {best_learning_rate})"] = best_model.predict(
        X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics in the eval metrics dictionary
    eval_metrics[model_name] = {
        'RMSE': best_rmse,
        'MAE': mae,
        'R-squared': r2,
        'SMAPE': smape_val,
        'Training Time (seconds)': training_time,
        'CPU Usage (MHz)': psutil.cpu_percent(),
        'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
    }

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('forecasts_792_Months_720tr_72t_1875_to_1941.csv', index=False)

# Save evaluation metrics to a CSV file
df_metrics = pd.DataFrame(eval_metrics).T
df_metrics.to_csv('eval_metrics_792_Months_1875_to_1941.csv')

# Print true values and forecasts in table format
df_true_values = pd.DataFrame(y_test, columns=['True Value'])
df_forecasts = pd.DataFrame(model_data)

print("True Values and Forecasts:")
print(pd.concat([df_true_values, df_forecasts], axis=1))


3/3 [==============================] - 1s 6ms/step
Model: LSTM (Units: 128, Learning Rate: 0.001)
RMSE: 12.88106
MAE: 10.20476
R-squared: 0.91273
SMAPE: 0.38743
Training Time: 45.72777 seconds
CPU Usage: 15.1 MHz
Memory Used: 15469.046875 MB

3/3 [==============================] - 0s 2ms/step
True Values and Forecasts:
    True Value  True Value  LSTM (Units: 128, Learning Rate: 0.001)
0         32.6        32.6                                28.873867
1         36.7        36.7                                32.284016
2         42.7        42.7                                36.535164
3         49.8        49.8                                42.759125
4         56.9        56.9                                50.098454
..         ...         ...                                      ...
66       111.3       111.3                               116.850945
67       107.7       107.7                               115.463951
68       103.2       103.2                               111.579468

# Deep learning 924 MONTHS (840 MONTHS training, 84 Testing) - Interval (1854-1931)

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1931 to 2008)
filtered_data = data[(data['Year'] >= 1931) & (data['Year'] <= 2008)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'Stacked LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'Bidirectional LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'GRU': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_units = 0
    best_dropout_rate = 0

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            if model_name == 'LSTM':
                model = create_lstm_model(units, dropout_rate)
            elif model_name == 'Stacked LSTM':
                model = create_stacked_lstm_model(units, dropout_rate)
            elif model_name == 'Bidirectional LSTM':
                model = create_bidirectional_lstm_model(units, dropout_rate)
            elif model_name == 'GRU':
                model = create_gru_model(units, dropout_rate)

            start_time = time.time()
            history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                y_train,
                                epochs=100,
                                batch_size=32,
                                verbose=0)
            training_time = time.time() - start_time

            predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, predictions)
            r2 = r2_score(y_test, predictions)
            smape_val = smape(y_test, predictions)

            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
                best_units = units
                best_dropout_rate = dropout_rate

            print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate})")
            print(f"RMSE: {rmse:.5f}")
            print(f"MAE: {mae:.5f}")
            print(f"R-squared: {r2:.5f}")
            print(f"SMAPE: {smape_val:.5f}")
            print(f"Training Time: {training_time:.5f} seconds")
            print(f"CPU Usage: {psutil.cpu_percent()} MHz")
            print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
            print()

    # Store the predictions in the dictionary
    model_data[f"{model_name} (Units: {best_units}, Dropout Rate: {best_dropout_rate})"] = best_model.predict(
        X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics in the eval metrics dictionary
    eval_metrics[model_name] = {
        'RMSE': best_rmse,
        'MAE': mae,
        'R-squared': r2,
        'SMAPE': smape_val,
        'Training Time (seconds)': training_time,
        'CPU Usage (MHz)': psutil.cpu_percent(),
        'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
    }

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('forecasts_1931_to_2008.csv', index=False)

# Save evaluation metrics to a CSV file
df_metrics = pd.DataFrame(eval_metrics).T
df_metrics.to_csv('eval_metrics_1931_to_2008.csv')

# Print true values and forecasts in table format
df_true_values = pd.DataFrame(y_test, columns=['True Value'])
df_forecasts = pd.DataFrame(model_data)

print("True Values and Forecasts:")
print(pd.concat([df_true_values, df_forecasts], axis=1))


# Deep learning 924 MONTHS (840 MONTHS training, 84 Testing) - Interval (1931-2008)

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import psutil
import io
import requests

# Set a random seed for reproducibility
np.random.seed(42)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number', 'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1931 to 2008)
filtered_data = data[(data['Year'] >= 1931) & (data['Year'] <= 2008)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values.reshape(-1, 1)

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size+test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the deep learning models to be evaluated
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_stacked_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(LSTM(units, input_shape=(1, X_train.shape[1]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_bidirectional_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(Bidirectional(LSTM(units), input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_gru_model(units, dropout_rate):
    model = Sequential()
    model.add(GRU(units, input_shape=(1, X_train.shape[1])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Hyperparameters for each model
hyperparameters = {
    'LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'Stacked LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'Bidirectional LSTM': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]},
    'GRU': {'units': [32, 64, 128], 'dropout_rate': [0.0, 0.2, 0.4]}
}

# Compute SMAPE (Symmetric Mean Absolute Percentage Error) for the models
def smape(y_true, y_pred):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))

# Create a dictionary to store the forecasts and actual values for each model
model_data = {'True Value': y_test.ravel()}

# Create a dictionary to store the evaluation metrics for each model
eval_metrics = {}

# Train and evaluate each deep learning model with hyperparameter tuning
for model_name, params in hyperparameters.items():
    best_rmse = np.inf
    best_model = None
    best_units = 0
    best_dropout_rate = 0

    for units in params['units']:
        for dropout_rate in params['dropout_rate']:
            if model_name == 'LSTM':
                model = create_lstm_model(units, dropout_rate)
            elif model_name == 'Stacked LSTM':
                model = create_stacked_lstm_model(units, dropout_rate)
            elif model_name == 'Bidirectional LSTM':
                model = create_bidirectional_lstm_model(units, dropout_rate)
            elif model_name == 'GRU':
                model = create_gru_model(units, dropout_rate)

            start_time = time.time()
            history = model.fit(X_train.reshape((X_train.shape[0], 1, X_train.shape[1])),
                                y_train,
                                epochs=100,
                                batch_size=32,
                                verbose=0)
            training_time = time.time() - start_time

            predictions = model.predict(X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()
            mse = mean_squared_error(y_test, predictions)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, predictions)
            r2 = r2_score(y_test, predictions)
            smape_val = smape(y_test, predictions)

            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
                best_units = units
                best_dropout_rate = dropout_rate

            print(f"Model: {model_name} (Units: {units}, Dropout Rate: {dropout_rate})")
            print(f"RMSE: {rmse:.5f}")
            print(f"MAE: {mae:.5f}")
            print(f"R-squared: {r2:.5f}")
            print(f"SMAPE: {smape_val:.5f}")
            print(f"Training Time: {training_time:.5f} seconds")
            print(f"CPU Usage: {psutil.cpu_percent()} MHz")
            print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")
            print()

    # Store the predictions in the dictionary
    model_data[f"{model_name} (Units: {best_units}, Dropout Rate: {best_dropout_rate})"] = best_model.predict(
        X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))).ravel()

    # Include evaluation metrics in the eval metrics dictionary
    eval_metrics[model_name] = {
        'RMSE': best_rmse,
        'MAE': mae,
        'R-squared': r2,
        'SMAPE': smape_val,
        'Training Time (seconds)': training_time,
        'CPU Usage (MHz)': psutil.cpu_percent(),
        'Memory Used (MB)': psutil.virtual_memory().used / 1024 / 1024
    }

# Save forecasts to a CSV file
df_data = pd.DataFrame(model_data)
df_data.to_csv('forecasts_1931_to_2008.csv', index=False)

# Save evaluation metrics to a CSV file
df_metrics = pd.DataFrame(eval_metrics).T
df_metrics.to_csv('eval_metrics_1931_to_2008.csv')

# Print true values and forecasts in table format
df_true_values = pd.DataFrame(y_test, columns=['True Value'])
df_forecasts = pd.DataFrame(model_data)

print("True Values and Forecasts:")
print(pd.concat([df_true_values, df_forecasts], axis=1))


In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import io
import requests
import time
import psutil
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values

# Normalize the target variable
#scaler = MinMaxScaler(feature_range=(0, 1))
#target_scaled = scaler.fit_transform(target.reshape(-1, 1)).flatten()

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size + test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Fit and evaluate Exponential Smoothing (ETS)
ets_model = ExponentialSmoothing(X_train)
start_time = time.time()
ets_model_fit = ets_model.fit()
training_time = time.time() - start_time

ets_predictions = ets_model_fit.forecast(test_size)  # Forecast the same length as test data

# Rescale the predictions back to the original scale
predictions = scaler.inverse_transform(ets_predictions.reshape(-1, 1)).flatten()

# Calculate the evaluation metrics
mse = mean_squared_error(y_test, predictions[:-1])  # Exclude the last predicted value
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions[:-1])
r2 = r2_score(y_test, predictions[:-1])
smape = 2 * np.mean(np.abs(y_test - predictions[:-1]) / (np.abs(y_test) + np.abs(predictions[:-1])))

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")
print(f"SMAPE: {smape:.5f}")
print(f"Training Time: {training_time:.5f} seconds")
print(f"CPU Usage: {psutil.cpu_percent()} MHz")
print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")


Exponential Smoothing (ETS) Results:
RMSE: 3623.03337
MAE: 3622.77098
R-squared: -6903.21514
SMAPE: 1.85874
Training Time: 0.00757 seconds
CPU Usage: 14.4 MHz
Memory Used: 6840.1171875 MB


C:\Users\Storm\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import io
import requests
import time
import psutil
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from itertools import product

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values

# Add a small positive value to handle non-positive values
target += 1e-8

# Normalize the target variable
scaler = MinMaxScaler(feature_range=(0, 1))
target_scaled = scaler.fit_transform(target.reshape(-1, 1)).flatten()

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target_scaled[:train_size]
test_data = target_scaled[train_size:train_size + test_size]

# Split into input and output variables
X_train = train_data[:-1]
y_train = train_data[1:]
X_test = test_data[:-1]
y_test = test_data[1:]

# Define the parameter grid for tuning
param_grid = {
    'trend': ['add', 'mul', None],
    'seasonal': ['add', 'mul', None],
    'seasonal_periods': [12],
    'damped': [True, False]
}

# Define variables to store best model and its performance
best_model = None
best_score = float('inf')

# Perform grid search for hyperparameter tuning
for params in product(*param_grid.values()):
    model_params = dict(zip(param_grid.keys(), params))
    model = ExponentialSmoothing(X_train, **model_params)
    try:
        model_fit = model.fit()
        predictions = model_fit.forecast(test_size)
        mse = mean_squared_error(y_test, predictions[:-1])
        if mse < best_score:
            best_model = model
            best_score = mse
    except:
        continue

# Fit the best model to the training data
start_time = time.time()
best_model_fit = best_model.fit()
training_time = time.time() - start_time

# Forecast with the best model
ets_predictions = best_model_fit.forecast(test_size)

# Rescale the predictions back to the original scale
predictions = scaler.inverse_transform(ets_predictions.reshape(-1, 1)).flatten()

# Calculate the evaluation metrics
mse = mean_squared_error(target[test_size:], predictions[:-1])  # Exclude the last predicted value
rmse = np.sqrt(mse)
mae = mean_absolute_error(target[test_size:], predictions[:-1])
r2 = r2_score(target[test_size:], predictions[:-1])
smape = 2 * np.mean(np.abs(target[test_size:] - predictions[:-1]) / (np.abs(target[test_size:]) + np.abs(predictions[:-1])))

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"Best Parameters: {best_model_fit.params}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")
print(f"SMAPE: {smape:.5f}")
print(f"Training Time: {training_time:.5f} seconds")
print(f"CPU Usage: {psutil.cpu_percent()} MHz")
print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")


In [1]:
import pandas as pd
import io
import requests
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size + test_size]

# Fit the Exponential Smoothing model to the training data
model = ExponentialSmoothing(train_data)
model_fit = model.fit()

# Forecast using the trained model
predictions = model_fit.forecast(test_size)

# Calculate the evaluation metrics
mse = ((predictions - test_data) ** 2).mean()
rmse = mse ** 0.5
mae = abs(predictions - test_data).mean()
r2 = 1 - (mse / ((test_data - test_data.mean()) ** 2).mean())

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")


C:\Users\Storm\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Storm\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


Exponential Smoothing (ETS) Results:
RMSE: 120.52697
MAE: 111.75694
R-squared: -6.13097


C:\Users\Storm\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(


In [3]:
import pandas as pd
import io
import requests
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size + test_size]

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'trend': ['add', 'mul', None],
    'seasonal': ['add', 'mul', None],
    'seasonal_periods': [12]
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')

# Perform manual hyperparameter tuning
for trend in param_grid['trend']:
    for seasonal in param_grid['seasonal']:
        model = ExponentialSmoothing(train_data, seasonal_periods=12, trend=trend, seasonal=seasonal,
                                     initialization_method='estimated')
        model_fit = model.fit()
        predictions = model_fit.forecast(test_size)
        mse = mean_squared_error(test_data, predictions)
        if mse < best_mse:
            best_model = model_fit
            best_mse = mse

# Forecast using the best model
predictions = best_model.forecast(test_size)

# Calculate the evaluation metrics
rmse = np.sqrt(best_mse)
mae = np.mean(np.abs(test_data - predictions))
r2 = 1 - (best_mse / np.var(test_data))

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"Best Parameters: trend={best_model.model.trend}, seasonal={best_model.model.seasonal}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")


C:\Users\Storm\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:80: RuntimeWarning: overflow encountered in matmul
  return err.T @ err


Exponential Smoothing (ETS) Results:
Best Parameters: trend=add, seasonal=add
RMSE: 67.38787
MAE: 54.06003
R-squared: -1.22917


In [5]:
import pandas as pd
import io
import requests
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size + test_size]

# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1)).flatten()
test_data_normalized = scaler.transform(test_data.reshape(-1, 1)).flatten()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'trend': ['add', None],
    'seasonal': ['add', None],
    'seasonal_periods': [12]
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')

# Perform manual hyperparameter tuning
for trend in param_grid['trend']:
    for seasonal in param_grid['seasonal']:
        model = ExponentialSmoothing(train_data_normalized, seasonal_periods=12, trend=trend, seasonal=seasonal,
                                     initialization_method='estimated')
        model_fit = model.fit()
        predictions_normalized = model_fit.forecast(test_size)
        predictions = scaler.inverse_transform(predictions_normalized.reshape(-1, 1)).flatten()
        mse = mean_squared_error(test_data, predictions)
        if mse < best_mse:
            best_model = model_fit
            best_mse = mse

# Forecast using the best model
predictions_normalized = best_model.forecast(test_size)
predictions = scaler.inverse_transform(predictions_normalized.reshape(-1, 1)).flatten()

# Calculate the evaluation metrics
rmse = np.sqrt(best_mse)
mae = np.mean(np.abs(test_data - predictions))
r2 = 1 - (best_mse / np.var(test_data))

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"Best Parameters: trend={best_model.model.trend}, seasonal={best_model.model.seasonal}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")


Exponential Smoothing (ETS) Results:
Best Parameters: trend=add, seasonal=add
RMSE: 67.39503
MAE: 54.06286
R-squared: -1.22964


In [7]:
import pandas as pd
import io
import requests
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data['13-Month Smoothed Monthly Total Sunspot Number'].values

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size + test_size]

# Normalize the data
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1)).flatten()
test_data_normalized = scaler.transform(test_data.reshape(-1, 1)).flatten()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'trend': ['add', None],
    'seasonal': ['add', None],
    'seasonal_periods': [12]
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')

# Perform manual hyperparameter tuning
for trend in param_grid['trend']:
    for seasonal in param_grid['seasonal']:
        model = ExponentialSmoothing(
            train_data_normalized,
            seasonal_periods=12,
            trend=trend,
            seasonal=seasonal,
            initialization_method='estimated',
        )
        model_fit = model.fit()
        predictions_normalized = model_fit.forecast(test_size)
        predictions = scaler.inverse_transform(predictions_normalized.reshape(-1, 1)).flatten()
        mse = mean_squared_error(test_data, predictions)
        if mse < best_mse:
            best_model = model_fit
            best_mse = mse

# Forecast using the best model
predictions_normalized = best_model.forecast(test_size)
predictions = scaler.inverse_transform(predictions_normalized.reshape(-1, 1)).flatten()

# Calculate the evaluation metrics
rmse = np.sqrt(best_mse)
mae = np.mean(np.abs(test_data - predictions))
r2 = 1 - (best_mse / np.var(test_data))

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"Best Parameters: trend={best_model.model.trend}, seasonal={best_model.model.seasonal}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")


Exponential Smoothing (ETS) Results:
Best Parameters: trend=add, seasonal=add
RMSE: 67.39503
MAE: 54.06286
R-squared: -1.22964


In [15]:
!pip install prophet


     -------------------------------------- 12.9/12.9 MB 195.1 kB/s eta 0:00:00
     -------------------------------------- 83.2/83.2 kB 166.7 kB/s eta 0:00:00
     -------------------------------------- 47.9/47.9 kB 240.0 kB/s eta 0:00:00
     ------------------------------------ 642.9/642.9 kB 295.6 kB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 204.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.4/1.4 MB 267.3 kB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732018 sha256=85c099ba1c0a2f43e6889cace92a4917818102fa88d2a2c1e47b8cb41fc60738
  Stored in directory: c:\users\storm\appdata\local\pip\cache\wheels\04\1f\e5\8dd0c661cd8d252817655dc14a84f7ae045d6616594145aa81
Successfully built pymeeus



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from prophet import Prophet
import io
import requests
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data[['Year', 'Month', '13-Month Smoothed Monthly Total Sunspot Number']]
target.columns = ['ds', 'month', 'y']

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size]
test_data = target[train_size:train_size + test_size]

# Normalize the data
scaler = MinMaxScaler()
train_data['y'] = scaler.fit_transform(train_data['y'].values.reshape(-1, 1)).flatten()
test_data['y'] = scaler.transform(test_data['y'].values.reshape(-1, 1)).flatten()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 1],
    'seasonality_prior_scale': [0.01, 0.1, 1, 10],
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')

# Perform manual hyperparameter tuning
for seasonality_mode in param_grid['seasonality_mode']:
    for changepoint_prior_scale in param_grid['changepoint_prior_scale']:
        for seasonality_prior_scale in param_grid['seasonality_prior_scale']:
            model = Prophet(
                seasonality_mode=seasonality_mode,
                changepoint_prior_scale=changepoint_prior_scale,
                seasonality_prior_scale=seasonality_prior_scale,
            )
            model.fit(train_data)
            future = model.make_future_dataframe(periods=test_size, freq='M')
            forecast = model.predict(future)
            predictions = forecast['yhat'].tail(test_size).values
            mse = mean_squared_error(test_data['y'], predictions)
            if mse < best_mse:
                best_model = model
                best_mse = mse

# Forecast using the best model
future = best_model.make_future_dataframe(periods=test_size, freq='M')
forecast = best_model.predict(future)
predictions = forecast['yhat'].tail(test_size).values
predictions = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()

# Calculate the evaluation metrics
rmse = np.sqrt(best_mse)
mae = np.mean(np.abs(test_data['y'] - predictions))
r2 = 1 - (best_mse / np.var(test_data['y']))

# Print the results
print("Prophet Results:")
print(f"Best Parameters: seasonality_mode={best_model.seasonality_mode}, "
      f"changepoint_prior_scale={best_model.changepoint_prior_scale}, "
      f"seasonality_prior_scale={best_model.seasonality_prior_scale}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")


C:\Users\Storm\AppData\Local\Temp/ipykernel_13384/3267026952.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['y'] = scaler.fit_transform(train_data['y'].values.reshape(-1, 1)).flatten()
C:\Users\Storm\AppData\Local\Temp/ipykernel_13384/3267026952.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['y'] = scaler.transform(test_data['y'].values.reshape(-1, 1)).flatten()
18:46:24 - cmdstanpy - INFO - Chain [1] start processing
18:46:24 - cmdstanpy - INFO - Chain [1] done processing
1

In [17]:
!pip install plotly

     --------------------------------------- 15.5/15.5 MB 70.3 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
import io
import requests
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data[['Year', 'Month', '13-Month Smoothed Monthly Total Sunspot Number']]
target.columns = ['ds', 'Month', 'y']

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size].copy()
test_data = target[train_size:train_size + test_size].copy()

# Normalize the data
scaler = MinMaxScaler()
train_data['y'] = scaler.fit_transform(train_data['y'].values.reshape(-1, 1)).flatten()
test_data['y'] = scaler.transform(test_data['y'].values.reshape(-1, 1)).flatten()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 1],
    'seasonality_prior_scale': [0.01, 0.1, 1, 10],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')
best_params = {}

# Perform manual hyperparameter tuning
for cp_scale in param_grid['changepoint_prior_scale']:
    for season_scale in param_grid['seasonality_prior_scale']:
        for season_mode in param_grid['seasonality_mode']:
            model = Prophet(
                growth='linear',
                seasonality_mode=season_mode,
                changepoint_prior_scale=cp_scale,
                seasonality_prior_scale=season_scale,
                holidays=None,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=False,
                interval_width=0.95,
            )
            model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
            model.fit(train_data)
            future = model.make_future_dataframe(periods=test_size, freq='M')
            forecast = model.predict(future)
            predictions = forecast['yhat'][-test_size:].values
            mse = mean_squared_error(test_data['y'], predictions)
            if mse < best_mse:
                best_model = model
                best_mse = mse
                best_params = {
                    'changepoint_prior_scale': cp_scale,
                    'seasonality_prior_scale': season_scale,
                    'seasonality_mode': season_mode
                }

# Forecast using the best model
future = best_model.make_future_dataframe(periods=test_size, freq='M')
forecast = best_model.predict(future)
predictions = forecast['yhat'][-test_size:].values
predictions = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()

# Calculate the evaluation metrics
rmse = np.sqrt(best_mse)
mae = np.mean(np.abs(test_data['y'] - predictions))
r2 = 1 - (best_mse / np.var(test_data['y']))

# Print the results
print("Prophet Results:")
print("Best Parameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")


19:05:12 - cmdstanpy - INFO - Chain [1] start processing
19:05:12 - cmdstanpy - INFO - Chain [1] done processing
19:05:12 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:05:12 - cmdstanpy - INFO - Chain [1] start processing
19:05:13 - cmdstanpy - INFO - Chain [1] done processing
19:05:13 - cmdstanpy - INFO - Chain [1] start processing
19:05:14 - cmdstanpy - INFO - Chain [1] done processing
19:05:14 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:05:14 - cmdstanpy - INFO - Chain [1] start processing
19:05:15 - cmdstanpy - INFO - Chain [1] done processing
19:05:15 - cmdstanpy - INFO - Chain [1] start processing
19:05:15 - cmdstanpy - INFO - Chain [1] done processing
19:05:15 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abn

Prophet Results:
Best Parameters:
changepoint_prior_scale: 0.01
seasonality_prior_scale: 10
seasonality_mode: multiplicative
RMSE: 0.37454
MAE: 89.63376
R-squared: -1.06572


In [16]:
import logging
import pandas as pd
import io
import requests
from sklearn.metrics import mean_squared_error
import numpy as np
from prophet import Prophet

# Set the logging level of cmdstanpy to suppress log messages
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data[['Year', 'Month', '13-Month Smoothed Monthly Total Sunspot Number']]
target.columns = ['ds', 'Month', 'y']

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size].copy()
test_data = target[train_size:train_size + test_size].copy()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'cp_scale': [0.001, 0.01, 0.1, 1],
    'season_scale': [0.01, 0.1, 1, 10],
    'season_mode': ['additive', 'multiplicative']
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')
best_params = {}

# Perform manual hyperparameter tuning
for cp_scale in param_grid['cp_scale']:
    for season_scale in param_grid['season_scale']:
        for season_mode in param_grid['season_mode']:
            model = Prophet(
                growth='linear',
                seasonality_mode=season_mode,
                changepoint_prior_scale=cp_scale,
                seasonality_prior_scale=season_scale,
                holidays=None,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=False,
                interval_width=0.95,
            )
            model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
            model.fit(train_data)
            future = model.make_future_dataframe(periods=test_size, freq='M')
            forecast = model.predict(future)
            predictions = forecast['yhat'][-test_size:].values
            mse = mean_squared_error(test_data['y'], predictions)
            if mse < best_mse:
                best_model = model
                best_mse = mse
                best_params = {
                    'cp_scale': cp_scale,
                    'season_scale': season_scale,
                    'season_mode': season_mode
                }

# Forecast using the best model
future = best_model.make_future_dataframe(periods=test_size, freq='M')
forecast = best_model.predict(future)
predictions = forecast['yhat'][-test_size:].values

# Calculate the evaluation metrics without scaling
rmse = np.sqrt(mean_squared_error(test_data['y'], predictions))
mae = np.mean(np.abs(test_data['y'] - predictions))
r2 = 1 - (best_mse / np.var(test_data['y']))

# Print the results
print("Prophet Results:")
print("Best Parameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")


19:15:30 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:15:31 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:15:33 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:15:34 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:15:36 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:15:37 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:15:38 - cmdstanpy - ERROR - Chain [1] error: error during pro

Prophet Results:
Best Parameters:
cp_scale: 0.01
season_scale: 1
season_mode: multiplicative
RMSE: 67.93616
MAE: 59.59697
R-squared: -1.26559


In [9]:
#!pip install fbprophet
!pip install prophet




[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install --upgrade pip


     ---------------------------------------- 2.1/2.1 MB 70.3 kB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\Storm\anaconda3\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import logging
import pandas as pd
import io
import requests
from sklearn.metrics import mean_squared_error
import numpy as np
from prophet import Prophet

# Set the logging level of cmdstanpy to suppress log messages
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)


# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data[['Year', 'Month', '13-Month Smoothed Monthly Total Sunspot Number']]
target.columns = ['ds', 'Month', 'y']

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size].copy()
test_data = target[train_size:train_size + test_size].copy()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 1],
    'seasonality_prior_scale': [0.01, 0.1, 1, 10],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')
best_params = {}

# Perform manual hyperparameter tuning
for cp_scale in param_grid['changepoint_prior_scale']:
    for season_scale in param_grid['seasonality_prior_scale']:
        for season_mode in param_grid['seasonality_mode']:
            model = Prophet(
                growth='linear',
                seasonality_mode=season_mode,
                changepoint_prior_scale=cp_scale,
                seasonality_prior_scale=season_scale,
                holidays=None,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=False,
                interval_width=0.95,
            )
            model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
            model.fit(train_data)
            future = model.make_future_dataframe(periods=test_size, freq='M')
            forecast = model.predict(future)
            predictions = forecast['yhat'][-test_size:].values
            mse = mean_squared_error(test_data['y'], predictions)
            if mse < best_mse:
                best_model = model
                best_mse = mse
                best_params = {
                    'changepoint_prior_scale': cp_scale,
                    'seasonality_prior_scale': season_scale,
                    'seasonality_mode': season_mode
                }

# Forecast using the best model
future = best_model.make_future_dataframe(periods=test_size, freq='M')
forecast = best_model.predict(future)
predictions = forecast['yhat'][-test_size:].values

# Calculate the evaluation metrics without scaling
rmse = np.sqrt(mean_squared_error(test_data['y'], predictions))
mae = np.mean(np.abs(test_data['y'] - predictions))
r2 = 1 - (best_mse / np.var(test_data['y']))

# Calculate SMAPE
def smape(actual, forecast):
    return np.mean((np.abs(actual - forecast) / (np.abs(actual) + np.abs(forecast))) * 200)

smape_value = smape(test_data['y'], predictions)

# Print the results
print("Exponential Smoothing (ETS) Results:")
print("Best Parameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")
print(f"SMAPE: {smape_value:.5f}")


19:20:46 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:20:47 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:20:48 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:20:49 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:20:50 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:20:52 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:20:53 - cmdstanpy - ERROR - Chain [1] error: error during pro

Exponential Smoothing (ETS) Results:
Best Parameters:
changepoint_prior_scale: 0.01
seasonality_prior_scale: 1
seasonality_mode: multiplicative
RMSE: 67.93616
MAE: 59.59697
R-squared: -1.26559
SMAPE: 51.30096


In [19]:
import logging
import pandas as pd
import io
import requests
from sklearn.metrics import mean_squared_error
import numpy as np
from prophet import Prophet
import time
import psutil

# Set the logging level of cmdstanpy to suppress log messages
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)


# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1875) & (data['Year'] <= 1941)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target = filtered_data[['Year', 'Month', '13-Month Smoothed Monthly Total Sunspot Number']]
target.columns = ['ds', 'Month', 'y']

# Split the data into training and testing sets
train_size = 720  # Number of months for training
test_size = 72  # Number of months for testing

train_data = target[:train_size].copy()
test_data = target[train_size:train_size + test_size].copy()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 1],
    'seasonality_prior_scale': [0.01, 0.1, 1, 10],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')
best_params = {}

# Perform manual hyperparameter tuning
start_time = time.time()
for cp_scale in param_grid['changepoint_prior_scale']:
    for season_scale in param_grid['seasonality_prior_scale']:
        for season_mode in param_grid['seasonality_mode']:
            model = Prophet(
                growth='linear',
                seasonality_mode=season_mode,
                changepoint_prior_scale=cp_scale,
                seasonality_prior_scale=season_scale,
                holidays=None,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=False,
                interval_width=0.95,
            )
            model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
            model.fit(train_data)
            future = model.make_future_dataframe(periods=test_size, freq='M')
            forecast = model.predict(future)
            predictions = forecast['yhat'][-test_size:].values
            mse = mean_squared_error(test_data['y'], predictions)
            if mse < best_mse:
                best_model = model
                best_mse = mse
                best_params = {
                    'changepoint_prior_scale': cp_scale,
                    'seasonality_prior_scale': season_scale,
                    'seasonality_mode': season_mode
                }

training_time = time.time() - start_time

# Forecast using the best model
future = best_model.make_future_dataframe(periods=test_size, freq='M')
forecast = best_model.predict(future)
predictions = forecast['yhat'][-test_size:].values

# Calculate the evaluation metrics without scaling
rmse = np.sqrt(mean_squared_error(test_data['y'], predictions))
mae = np.mean(np.abs(test_data['y'] - predictions))
r2 = 1 - (best_mse / np.var(test_data['y']))

# Calculate SMAPE
def smape(actual, forecast):
    return np.mean((np.abs(actual - forecast) / (np.abs(actual) + np.abs(forecast))) * 200)

smape_value = smape(test_data['y'], predictions)

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"Best Parameters: {best_params}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")
print(f"SMAPE: {smape_value:.5f}")
print(f"Training Time: {training_time:.5f} seconds")
print(f"CPU Usage: {psutil.cpu_percent()} MHz")
print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")


19:23:32 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:23:33 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:23:35 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:23:36 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:23:37 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:23:39 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
19:23:40 - cmdstanpy - ERROR - Chain [1] error: error during pro

Exponential Smoothing (ETS) Results:
Best Parameters: {'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 1, 'seasonality_mode': 'multiplicative'}
RMSE: 67.93616
MAE: 59.59697
R-squared: -1.26559
SMAPE: 51.30096
Training Time: 25.57862 seconds
CPU Usage: 17.1 MHz
Memory Used: 7342.16796875 MB


In [27]:
import logging
import pandas as pd
import io
import requests
from sklearn.metrics import mean_squared_error
import numpy as np
from prophet import Prophet
import time
import psutil

# Set the logging level of cmdstanpy to suppress log messages
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1854) & (data['Year'] <= 1931)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target_column = '13-Month Smoothed Monthly Total Sunspot Number'
target = filtered_data[['Year', 'Month', target_column]]
target.columns = ['ds', 'Month', 'y']

# Split the data into training and testing sets
train_size = 840  # Number of months for training
test_size = 84  # Number of months for testing

train_data = target[:train_size].copy()
test_data = target[train_size:train_size + test_size].copy()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 1],
    'seasonality_prior_scale': [0.01, 0.1, 1, 10],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')
best_params = {}

# Perform manual hyperparameter tuning
start_time = time.time()
for cp_scale in param_grid['changepoint_prior_scale']:
    for season_scale in param_grid['seasonality_prior_scale']:
        for season_mode in param_grid['seasonality_mode']:
            model = Prophet(
                growth='linear',
                seasonality_mode=season_mode,
                changepoint_prior_scale=cp_scale,
                seasonality_prior_scale=season_scale,
                holidays=None,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=False,
                interval_width=0.95,
            )
            model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
            model.fit(train_data)
            future = model.make_future_dataframe(periods=test_size, freq='M')
            forecast = model.predict(future)
            predictions = forecast['yhat'][-test_size:].values
            mse = mean_squared_error(test_data['y'], predictions)
            if mse < best_mse:
                best_model = model
                best_mse = mse
                best_params = {
                    'changepoint_prior_scale': cp_scale,
                    'seasonality_prior_scale': season_scale,
                    'seasonality_mode': season_mode
                }

training_time = time.time() - start_time

# Forecast using the best model
future = best_model.make_future_dataframe(periods=test_size, freq='M')
forecast = best_model.predict(future)
predictions = forecast['yhat'][-test_size:].values

# Calculate the evaluation metrics without scaling
rmse = np.sqrt(mean_squared_error(test_data['y'], predictions))
mae = np.mean(np.abs(test_data['y'] - predictions))
r2 = 1 - (best_mse / np.var(test_data['y']))

# Calculate SMAPE
def smape(actual, forecast):
    return np.mean((np.abs(actual - forecast) / (np.abs(actual) + np.abs(forecast))) * 200)

smape_value = smape(test_data['y'], predictions)

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"Best Parameters: {best_params}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")
print(f"SMAPE: {smape_value:.5f}")
print(f"Training Time: {training_time:.5f} seconds")
print(f"CPU Usage: {psutil.cpu_percent()} MHz")
print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")


21:18:10 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
21:18:11 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
21:18:13 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
21:18:15 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.


Exponential Smoothing (ETS) Results:
Best Parameters: {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 0.1, 'seasonality_mode': 'additive'}
RMSE: 34.95585
MAE: 31.94644
R-squared: -0.06092
SMAPE: 41.29613
Training Time: 17.77968 seconds
CPU Usage: 23.8 MHz
Memory Used: 6943.9921875 MB


In [28]:
import logging
import pandas as pd
import io
import requests
from sklearn.metrics import mean_squared_error
import numpy as np
from prophet import Prophet
import time
import psutil

# Set the logging level of cmdstanpy to suppress log messages
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

# Fetch the data from the website
url = 'https://www.sidc.be/SILSO/INFO/snmstotcsv.php'
response = requests.get(url)
data = pd.read_csv(io.StringIO(response.text), delimiter=';', header=None)
data.columns = ['Year', 'Month', 'Decimal date', '13-Month Smoothed Monthly Total Sunspot Number',
                'Standard deviation', 'Number of observations', 'Definitive/Provisional']

# Filter the data for the desired interval (1875 to 1941)
filtered_data = data[(data['Year'] >= 1931) & (data['Year'] <= 2008)].copy()

# Select the '13-Month Smoothed Monthly Total Sunspot Number' column as the target variable
target_column = '13-Month Smoothed Monthly Total Sunspot Number'
target = filtered_data[['Year', 'Month', target_column]]
target.columns = ['ds', 'Month', 'y']

# Split the data into training and testing sets
train_size = 840  # Number of months for training
test_size = 84  # Number of months for testing

train_data = target[:train_size].copy()
test_data = target[train_size:train_size + test_size].copy()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 1],
    'seasonality_prior_scale': [0.01, 0.1, 1, 10],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Initialize variables to store best model and its performance
best_model = None
best_mse = float('inf')
best_params = {}

# Perform manual hyperparameter tuning
start_time = time.time()
for cp_scale in param_grid['changepoint_prior_scale']:
    for season_scale in param_grid['seasonality_prior_scale']:
        for season_mode in param_grid['seasonality_mode']:
            model = Prophet(
                growth='linear',
                seasonality_mode=season_mode,
                changepoint_prior_scale=cp_scale,
                seasonality_prior_scale=season_scale,
                holidays=None,
                daily_seasonality=False,
                weekly_seasonality=False,
                yearly_seasonality=False,
                interval_width=0.95,
            )
            model.add_seasonality(name='yearly', period=365.25, fourier_order=10)
            model.fit(train_data)
            future = model.make_future_dataframe(periods=test_size, freq='M')
            forecast = model.predict(future)
            predictions = forecast['yhat'][-test_size:].values
            mse = mean_squared_error(test_data['y'], predictions)
            if mse < best_mse:
                best_model = model
                best_mse = mse
                best_params = {
                    'changepoint_prior_scale': cp_scale,
                    'seasonality_prior_scale': season_scale,
                    'seasonality_mode': season_mode
                }

training_time = time.time() - start_time

# Forecast using the best model
future = best_model.make_future_dataframe(periods=test_size, freq='M')
forecast = best_model.predict(future)
predictions = forecast['yhat'][-test_size:].values

# Calculate the evaluation metrics without scaling
rmse = np.sqrt(mean_squared_error(test_data['y'], predictions))
mae = np.mean(np.abs(test_data['y'] - predictions))
r2 = 1 - (best_mse / np.var(test_data['y']))

# Calculate SMAPE
def smape(actual, forecast):
    return np.mean((np.abs(actual - forecast) / (np.abs(actual) + np.abs(forecast))) * 200)

smape_value = smape(test_data['y'], predictions)

# Print the results
print("Exponential Smoothing (ETS) Results:")
print(f"Best Parameters: {best_params}")
print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")
print(f"R-squared: {r2:.5f}")
print(f"SMAPE: {smape_value:.5f}")
print(f"Training Time: {training_time:.5f} seconds")
print(f"CPU Usage: {psutil.cpu_percent()} MHz")
print(f"Memory Used: {psutil.virtual_memory().used / 1024 / 1024} MB")


Exponential Smoothing (ETS) Results:
Best Parameters: {'changepoint_prior_scale': 1, 'seasonality_prior_scale': 0.01, 'seasonality_mode': 'additive'}
RMSE: 43.00714
MAE: 37.29695
R-squared: 0.46557
SMAPE: 58.73349
Training Time: 13.69575 seconds
CPU Usage: 31.0 MHz
Memory Used: 6976.8515625 MB


In [ ]:
# Plot the predictions
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(y_train) + 1), y_train, label='Actual (Training)')
plt.plot(range(len(y_train), len(y_train) + len(y_test)), y_test, label='Actual (Testing)')
plt.plot(range(1, len(y_train) + 1), train_predictions, label='Predicted (Training)')
plt.plot(range(len(y_train), len(y_train) + len(y_test)), test_predictions, label='Predicted (Testing)')
plt.title('ETS Model Performance')
plt.xlabel('Months')
plt.ylabel('Sunspot Number')
plt.legend()
plt.show()